#📌 Extracción

In [4]:
import pandas as pd

# URL de los datos en formato JSON
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

# Cargar los datos en un DataFrame
datos = pd.read_json(url)

# Mostrar las primeras filas del DataFrame para verificar la carga
datos.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


### Conociendo el Conjunto de Datos

Ahora que los datos han sido extraídos y cargados en un DataFrame, el siguiente paso es familiarizarnos con su estructura. Esto implica entender qué información contiene cada columna, los tipos de datos presentes y la forma general del dataset. Esta exploración inicial es crucial para planificar los pasos de limpieza y transformación.

#### Exploración de Columnas y Tipos de Datos

Vamos a utilizar el método `.info()` de Pandas para obtener un resumen técnico de nuestro DataFrame. `.info()` nos dará una visión general de las columnas, la cantidad de valores no nulos y el uso de memoria.

In [5]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


#### Primeras Observaciones

De la salida anterior, notamos algo muy importante: las columnas `customer`, `phone`, `internet`, y `account` son de tipo `object`. Al observar las primeras filas con `.head()`, vimos que contienen datos anidados, similares a un diccionario o JSON. 

Esto significa que una parte crucial de la fase de **Transformación** será "desanidar" o "normalizar" estas columnas para que cada pieza de información (como el género del cliente, si tiene servicio de teléfono, el tipo de contrato, etc.) se convierta en su propia columna en el DataFrame. La columna `Churn` parece ser nuestro objetivo principal para el análisis, indicando si un cliente ha cancelado el servicio o no.

#### Contenido del Diccionario de Datos

Para facilitar la consulta, a continuación se muestra el contenido del diccionario de datos:

```
- `customerID`: número de identificación único de cada cliente
- `Churn`: si el cliente dejó o no la empresa
- `gender`: género (masculino y femenino)
- `SeniorCitizen`: información sobre si un cliente tiene o no una edad igual o mayor a 65 años
- `Partner`: si el cliente tiene o no una pareja
- `Dependents`: si el cliente tiene o no dependientes
- `tenure`: meses de contrato del cliente
- `PhoneService`: suscripción al servicio telefónico
- `MultipleLines`: suscripción a más de una línea telefónica
- `InternetService`: suscripción a un proveedor de internet
- `OnlineSecurity`: suscripción adicional de seguridad en línea
- `OnlineBackup`: suscripción adicional de respaldo en línea
- `DeviceProtection`: suscripción adicional de protección del dispositivo
- `TechSupport`: suscripción adicional de soporte técnico, menor tiempo de espera
- `StreamingTV`: suscripción de televisión por cable
- `StreamingMovies`: suscripción de streaming de películas
- `Contract`: tipo de contrato
- `PaperlessBilling`: si el cliente prefiere recibir la factura en línea
- `PaymentMethod`: forma de pago
- `Charges.Monthly`: total de todos los servicios del cliente por mes
- `Charges.Total`: total gastado por el cliente
```

#### Inspección de las Columnas Anidadas

Como se observó, cuatro columnas (`customer`, `phone`, `internet`, `account`) contienen datos anidados. Antes de transformarlos, es crucial inspeccionar su estructura para entender los campos que contienen. Analizaremos el primer registro de cada una como muestra.

In [6]:
datos["customer"][0]

{'gender': 'Female',
 'SeniorCitizen': 0,
 'Partner': 'Yes',
 'Dependents': 'Yes',
 'tenure': 9}

In [7]:
datos["phone"][0]

{'PhoneService': 'Yes', 'MultipleLines': 'No'}

In [8]:
datos["internet"][0]

{'InternetService': 'DSL',
 'OnlineSecurity': 'No',
 'OnlineBackup': 'Yes',
 'DeviceProtection': 'No',
 'TechSupport': 'Yes',
 'StreamingTV': 'Yes',
 'StreamingMovies': 'No'}

In [9]:
datos["account"][0]

{'Contract': 'One year',
 'PaperlessBilling': 'Yes',
 'PaymentMethod': 'Mailed check',
 'Charges': {'Monthly': 65.6, 'Total': '593.3'}}

#### Conclusiones de la Inspección

Esta inspección confirma que estas columnas son efectivamente diccionarios que deberán ser expandidos. Las claves de estos diccionarios (`gender`, `SeniorCitizen`, `Contract`, `PaymentMethod`, etc.) se convertirán en las nuevas columnas de nuestro DataFrame.

Todas estas variables son candidatas potenciales para analizar la evasión de clientes (`Churn`). En la fase de **Transformación**, nuestro primer paso será normalizar estos datos para poder utilizarlos en el análisis.

#🔧 Transformación

#📊 Carga y análisis

#📄Informe final